I chose the state of florida to view the last week of covid data for. The 7 day average for cases and deaths in florida is stable as there have been 0 reported deaths and cases in the last 7 days.

In [1]:
import pandas as pd

df = pd.read_csv('president_county_candidate.csv')

df

,state,county,candidate,party,total_votes,won
0,Delaware,Kent County,Joe Biden,DEM,44552,True
1,Delaware,Kent County,Donald Trump,REP,41009,False
2,Delaware,Kent County,Jo Jorgensen,LIB,1044,False
3,Delaware,Kent County,Howie Hawkins,GRN,420,False
4,Delaware,New Castle County,Joe Biden,DEM,195034,True
...,...,...,...,...,...,...
32172,Arizona,Maricopa County,Write-ins,WRI,1331,False
32173,Arizona,Mohave County,Donald Trump,REP,78535,True
32174,Arizona,Mohave County,Joe Biden,DEM,24831,False
32175,Arizona,Mohave County,Jo Jorgensen,LIB,1302,False


In [14]:
# Load election data and super COVID data
election_df = pd.read_csv('president_county_candidate.csv')
super_df = pd.read_csv('super_covid19_data.csv')

# Step 1: Filter for Joe Biden and Donald Trump only
filtered_election_df = election_df[election_df['candidate'].isin(['Joe Biden', 'Donald Trump'])]

# Step 2: Keep only the relevant columns including total_votes
filtered_election_df = filtered_election_df[['state', 'county', 'party', 'won', 'total_votes']]

# Step 3: Clean up the county and state names in the election data
filtered_election_df['county'] = filtered_election_df['county'].str.strip().str.lower()
filtered_election_df['state'] = filtered_election_df['state'].str.strip().str.lower()

# Step 4: Create a dictionary to map state abbreviations to full state names
state_abbrev_to_name = {
    'AL': 'alabama', 'AK': 'alaska', 'AZ': 'arizona', 'AR': 'arkansas', 'CA': 'california', 'CO': 'colorado',
    'CT': 'connecticut', 'DE': 'delaware', 'FL': 'florida', 'GA': 'georgia', 'HI': 'hawaii', 'ID': 'idaho',
    'IL': 'illinois', 'IN': 'indiana', 'IA': 'iowa', 'KS': 'kansas', 'KY': 'kentucky', 'LA': 'louisiana',
    'ME': 'maine', 'MD': 'maryland', 'MA': 'massachusetts', 'MI': 'michigan', 'MN': 'minnesota', 'MS': 'mississippi',
    'MO': 'missouri', 'MT': 'montana', 'NE': 'nebraska', 'NV': 'nevada', 'NH': 'new hampshire', 'NJ': 'new jersey',
    'NM': 'new mexico', 'NY': 'new york', 'NC': 'north carolina', 'ND': 'north dakota', 'OH': 'ohio', 'OK': 'oklahoma',
    'OR': 'oregon', 'PA': 'pennsylvania', 'RI': 'rhode island', 'SC': 'south carolina', 'SD': 'south dakota',
    'TN': 'tennessee', 'TX': 'texas', 'UT': 'utah', 'VT': 'vermont', 'VA': 'virginia', 'WA': 'washington',
    'WV': 'west virginia', 'WI': 'wisconsin', 'WY': 'wyoming'
}

# Step 5: Apply this dictionary to the COVID data to convert state abbreviations to full names
super_df['State'] = super_df['State'].map(state_abbrev_to_name).fillna(super_df['State'])

# Step 6: Select the relevant columns from the COVID data and clean up county and state names
columns_to_keep = ['countyFIPS', 'County Name_x', 'State'] + [col for col in super_df.columns if '_cases' in col or '_deaths' in col]
covid_df = super_df[columns_to_keep]
covid_df.loc[:, 'County Name_x'] = covid_df['County Name_x'].str.strip().str.lower()
covid_df.loc[:, 'State'] = covid_df['State'].str.strip().str.lower()


# Step 7: Merge the COVID data and election data on both county and state
merged_df = pd.merge(covid_df, filtered_election_df, left_on=['County Name_x', 'State'], right_on=['county', 'state'])

# Step 8: Sort by total_votes to ensure highest votes come first
merged_df = merged_df.sort_values(by=['county', 'total_votes'], ascending=False)

# Step 9: Drop duplicate counties, keeping only the one with the highest total_votes
merged_df_highest_votes = merged_df.drop_duplicates(subset=['countyFIPS', 'County Name_x', 'State'], keep='first')

# Step 10: Inspect the final result to ensure it reflects the highest votes per county and state
print(merged_df_highest_votes[['county', 'state', 'party', 'won', 'total_votes']].head())

# Export to CSV
merged_df_highest_votes.to_csv('merged_winners_election_covid_data.csv', index=False)


              county         state party   won  total_votes
4672  ziebach county  south dakota   DEM  True          481
5370   zavala county         texas   DEM  True         2864
5368   zapata county         texas   REP  True         2033
162      yuma county       arizona   REP  True        36534
550      yuma county      colorado   REP  True         4107
